# Forecast-Based Portfolio Optimization Using Time Series and Volatility Modeling 

**Purpose of the project**

to provide scientific and data-driven guidance to investors by in-depth analyzing the historical performance of Borsa Istanbul (BIST) stocks and developing forecast-based portfolio management strategies.


**The project progresses in three main phases:**

**1.** Historical Return Analysis

**2.** 90-Day Forecast Modeling

**3.** Optimum Portfolio Management

**NOTE:**

In this study, a 1,000,000 TL investment portfolio was created using shares from three major 
and representative companies listed on Borsa Istanbul: Turkish Airlines (THYAO), Akbank 
(AKBNK), and Arçelik (ARCLK). These companies were chosen because they represent 
different sectors, have a significant influence on the Turkish economy, and are widely 
followed by investors.

**Packages**

In [ ]:
!pip install numpy==1.24.4 --quiet
!pip install pmdarima==2.0.3 --quiet
!pip install yfinance arch seaborn --quiet
!pip install yfinance --quiet
!pip install seaborn --quiet
!pip install pmdarima --quiet
!pip install arch --quiet




In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import os
import statsmodels.api as sm
import pmdarima as pm
from datetime import timedelta
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from arch import arch_model
import scipy.stats as stats
from statsmodels.stats.diagnostic import acorr_ljungbox


## Correlation Analysis

In [ ]:
all_stocks_to_consider = [
'THYAO.IS', # Turkish Airlines - Transportation
'FROTO.IS', # Ford Otosan - Automotive
'BIMAS.IS', # Bim United Stores - Retail
'AKBNK.IS', # Akbank - Banking
'EREGL.IS', # Ereğli Iron and Steel - Iron and Steel
'SAHOL.IS', # Sabancı Holding - Holding
'ASELS.IS', # Aselsan - Defense Industry
'TUPRS.IS', # Tüpraş - Oil Refinery
'GARAN.IS', # Garanti BBVA - Banking
'ISCTR.IS', # İş Bankası C - Banking
'KOZAL.IS', # Koza Gold Operations - Mining
'PGSUS.IS', # Pegasus Air Transportation - Transportation
'TCELL.IS', # Turkcell - Telecommunications
'SISE.IS', # Şişecam - Glass and Chemicals
'VESTL.IS', # Vestel Electronics - White Goods/Electronics
'ARCLK.IS', # Arçelik - White Goods
'KCHOL.IS', # Koç Holding - Holding
'ULKER.IS', # Ülker Bisküvi - Food
'HALKB.IS', # Halkbank - Banking
'VAKBN.IS', # VakıfBank - Banking
'PETKM.IS' # Petkim - Petrochemicals
]

start_date = '2022-01-01'
end_date = '2024-12-31'

In [ ]:
full_data = yf.download(all_stocks_to_consider, start=start_date, end=end_date, auto_adjust=False)['Adj Close'].dropna()
full_data.dropna(inplace=True)

full_returns = full_data.pct_change().dropna()

plt.figure(figsize=(14, 12))
sns.heatmap(full_returns.corr(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title("Potential Candidate Stock Return Correlation")
plt.show()

historical_volatilities = full_returns.std() * (252**0.5)
print("\nHistorical Annual Volatilities:\n", historical_volatilities.sort_values(ascending=False))

As shown in the Correlation Table, there is moderate to low dispersion between the ranges in terms of the trend of returns. Lower correlations help enhance portfolio diversification, which in turn reduces systematic risk. Therefore, THYAO and AKBNK (-0.46) and ARCLK and AKBNK (-0.47) were selected for analysis.

In [ ]:
stocks = ['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]

data = yf.download(stocks, start=start_date, end=end_date, auto_adjust=False)['Adj Close'].dropna()
data.dropna(inplace=True)

returns = data.pct_change().dropna()

stats_df = pd.DataFrame({
    'Mean (%)'      : returns[['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]].mean()*100,
    'Standard Deviation (%)'     : returns[['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]].std()*100,
    'Skewness'          : returns[['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]].apply(stats.skew),
    'Kurtosis'          : returns[['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]].apply(stats.kurtosis)
}).T
print(stats_df)


## Daily Return & Return Correlation








In [ ]:
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]


stocks = ['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]
start_date = '2022-01-01'
end_date = '2024-12-31'


data = yf.download(stocks, start=start_date, end=end_date, auto_adjust=False)['Adj Close'].dropna()
data.dropna(inplace=True)


data.plot(title='Share Prices (2022–2024)')
plt.ylabel('Price (TL)')
plt.xlabel('Date')
plt.show()


returns = data.pct_change().dropna()


returns.plot(title='Daily Returns')
plt.ylabel('Rate')
plt.xlabel('Date')
plt.show()


sns.heatmap(returns.corr(), annot=True, cmap='coolwarm')
plt.title("Return Correlation")
plt.show()


In [ ]:
if not os.path.exists('figures'):
    os.makedirs('figures')

for stock in ['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]:

    plt.figure()
    sns.histplot(returns[stock], bins=40, kde=True, stat='density', color='skyblue')
    plt.title(f'{stock} – Daily Return Histogram and Normal Curve')
    plt.xlabel('Daily Return')
    plt.ylabel('Density')
    plt.grid(True)

    plt.savefig(f"figures/sekil_4_hist_{stock}.png", dpi=300)
    plt.show()

In [ ]:
window = 20
for stock in ['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]:
    plt.figure()
    plt.plot(data[stock], label='Price', color='gray')
    plt.plot(data[stock].rolling(window).mean(), label=f'{window} Daily Avg.', color='blue')
    plt.plot(data[stock].rolling(window).std(), label=f'{window} Daily Std (Volatility)', color='red')
    plt.title(f'{stock} – {window} Daily Avg and Volatility')
    plt.xlabel('Date')
    plt.ylabel('Price / Std')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"figures/sekil_5_rolling_{stock}.png", dpi=300)
    plt.show()


## Regression Between Selected Stoks

In [ ]:

Y = returns['ARCLK.IS']

X = returns [['THYAO.IS', 'AKBNK.IS']]

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()
print(model.summary())


In [ ]:
bist = yf.download('XU100.IS', start=start_date, end=end_date, auto_adjust=True)['Close']
bist_returns = bist.pct_change().dropna()


In [ ]:
# Regression- BIST100

for stock in ['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]:
    plt.figure()

    aligned_returns = returns[stock].reindex(bist_returns.index).dropna()
    sns.regplot(x=bist_returns.loc[aligned_returns.index], y=aligned_returns,
                line_kws={"color": "red"}, scatter_kws={"alpha": 0.5})
    plt.title(f'{stock} – Return Regression with BIST100')
    plt.xlabel('BIST100 Daily Return')
    plt.ylabel(f'{stock} Daily Return')
    plt.grid(True)
    plt.savefig(f"figures/sekil_6_regression_{stock}.png", dpi=300)
    plt.show()

In [ ]:

regression_start_date = '2022-01-01'
regression_end_date = '2024-12-31'

bist_close   = yf.download('XU100.IS', start=regression_start_date, end=regression_end_date, auto_adjust=True)['Close']
bist_returns = bist_close.pct_change().dropna()


Y = returns['THYAO.IS']


X = bist_returns.rename(columns={'XU100.IS': 'BIST100'})

regression_data = pd.concat([Y, X], axis=1).dropna()

Y_aligned = regression_data['THYAO.IS']
X_aligned = regression_data['BIST100']

X_aligned = sm.add_constant(X_aligned)

reg_THYAO = sm.OLS(Y_aligned, X_aligned).fit()
print(reg_THYAO.summary())      # tablo
print("Alpha  :", reg_THYAO.params['const'])
print("Beta   :", reg_THYAO.params['BIST100'])
print("R²     :", reg_THYAO.rsquared)
print("p‑değ. :", reg_THYAO.pvalues['BIST100'])

In [ ]:
regression_start_date = '2022-01-01'
regression_end_date = '2024-12-31'
bist_close   = yf.download('XU100.IS', start=regression_start_date, end=regression_end_date, auto_adjust=True)['Close']
bist_returns = bist_close.pct_change().dropna()

Y = returns['AKBNK.IS']

X = bist_returns.rename(columns={'XU100.IS': 'BIST100'})

regression_data = pd.concat([Y, X], axis=1).dropna()

Y_aligned = regression_data['AKBNK.IS']
X_aligned = regression_data['BIST100']

X_aligned = sm.add_constant(X_aligned)

reg_AKBNK = sm.OLS(Y_aligned, X_aligned).fit()
print(reg_AKBNK.summary())
print("Alpha  :", reg_AKBNK.params['const'])
print("Beta   :", reg_AKBNK.params['BIST100'])
print("R²     :", reg_AKBNK.rsquared)
print("p‑değ. :", reg_AKBNK.pvalues['BIST100'])

In [ ]:
regression_start_date = '2022-01-01'
regression_end_date = '2024-12-31'

bist_close   = yf.download('XU100.IS', start=regression_start_date, end=regression_end_date, auto_adjust=True)['Close']
bist_returns = bist_close.pct_change().dropna()

Y = returns['ARCLK.IS']

X = bist_returns.rename(columns={'XU100.IS': 'BIST100'})

regression_data = pd.concat([Y, X], axis=1).dropna()

Y_aligned = regression_data['ARCLK.IS']
X_aligned = regression_data['BIST100']

X_aligned = sm.add_constant(X_aligned)

reg_ARCLK = sm.OLS(Y_aligned, X_aligned).fit()
print(reg_ARCLK.summary())
print("Alpha  :", reg_ARCLK.params['const'])
print("Beta   :", reg_ARCLK.params['BIST100'])
print("R²     :", reg_ARCLK.rsquared)
print("p‑değ. :", reg_ARCLK.pvalues['BIST100'])

##  90-Day Forecast Modeling

Time series analyses were performed on the daily closing prices of three selected stocks and a 90-day forecast was produced for each stock.

Before the forecast process,
* the stationarity test was performed with the ADF test,
* the appropriate SARIMA model was selected with auto.arima
* ACF and PACF graphs were drawn
* the quality of the model was evaluated with the Ljung-Box test applied to the residuals. 

### ADF Test

In [ ]:
pd.options.display.float_format = "{:.4f}".format
for s in ['THYAO.IS', 'AKBNK.IS', 'ARCLK.IS' ]:
    adf_stat, p, lags, obs, crit, _ = adfuller(returns[s])
    print(f"\n{s}  ADF Test")
    print("Test statistics :", adf_stat)
    print(f"p-değeri         : {p}")
    for k, v in crit.items():
        print(f"Critical value {k}: {v}")


### ARIMA Model Training

In [ ]:

thy_price = data['THYAO.IS'].dropna()

log_thy = np.log(thy_price)

model_thy = pm.auto_arima(log_thy,
                          seasonal=False,
                          stepwise=True,
                          suppress_warnings=True,
                          max_p=5, max_q=5, max_d=2,
                          trace=True)

n_days = 90
forecast_log = model_thy.predict(n_periods=n_days)

forecast_thy = np.exp(forecast_log)

last_date = thy_price.index[-1]
forecast_dates = pd.date_range(start=last_date + timedelta(days=1), periods=n_days, freq='B')

print("THYAO - ARIMA Model Summary:")
print(model_thy.summary())


plt.figure(figsize=(10, 5))
plt.plot(thy_price, label='Actual Data')
plt.plot(forecast_dates, forecast_thy, label='90 Day Forecast', color='red')
plt.title("THYAO - ARIMA Forecast (Log Transformed)")
plt.xlabel("Date")
plt.ylabel("Price (TL)")
plt.legend()
plt.show()

thy_price = data['THYAO.IS'].dropna()

log_thy = np.log(thy_price)

diff_log_thy = log_thy.diff().dropna()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

plot_acf(diff_log_thy, ax=axes[0], lags=30)
axes[0].set_title("THYAO - ACF (Log + 1. Fark)")

plot_pacf(diff_log_thy, ax=axes[1], lags=30, method='ywm')
axes[1].set_title("THYAO - PACF (Log + 1. Fark)")

plt.tight_layout()
plt.show()



In [ ]:


arclk_price = data['ARCLK.IS'].dropna()
log_arclk = np.log(arclk_price)

model_arclk = pm.auto_arima(log_arclk,
                            seasonal=False,
                            stepwise=True,
                            suppress_warnings=True,
                            max_p=5, max_q=5, max_d=2,
                            trace=True)

n_days = 90
forecast_log_arclk = model_arclk.predict(n_periods=n_days)
forecast_arclk = np.exp(forecast_log_arclk)

last_date = arclk_price.index[-1]
forecast_dates = pd.date_range(start=last_date + timedelta(days=1), periods=n_days, freq='B')

print("ARCLK - ARIMA Model Summary:")
print(model_arclk.summary())

plt.figure(figsize=(10, 5))
plt.plot(arclk_price, label='Actual Data')
plt.plot(forecast_dates, forecast_arclk, label='90 Day Forecast', color='red')
plt.title("ARCLK - ARIMA Forecast (Log Transformed)")
plt.xlabel("Date")
plt.ylabel("Price (TL)")
plt.legend()
plt.tight_layout()
plt.show()

diff_log_arclk = log_arclk.diff().dropna()
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
plot_acf(diff_log_arclk, ax=axes[0], lags=30)
axes[0].set_title("ARCLK - ACF (Log + 1. Fark)")
plot_pacf(diff_log_arclk, ax=axes[1], lags=30, method='ywm')
axes[1].set_title("ARCLK - PACF (Log + 1. Fark)")
plt.tight_layout()
plt.show()


In [ ]:
akbnk_price = data['AKBNK.IS'].dropna()
log_akbnk = np.log(akbnk_price)

model_akbnk = pm.auto_arima(log_akbnk,
                            seasonal=False,
                            stepwise=True,
                            suppress_warnings=True,
                            max_p=5, max_q=5, max_d=2,
                            trace=True)

forecast_log_akbnk = model_akbnk.predict(n_periods=90)
forecast_akbnk = np.exp(forecast_log_akbnk)

last_date = akbnk_price.index[-1]
forecast_dates = pd.date_range(start=last_date + timedelta(days=1), periods=90, freq='B')

print("AKBNK - ARIMA Model Summary:")
print(model_akbnk.summary())

plt.figure(figsize=(10, 5))
plt.plot(akbnk_price, label='Actual Data')
plt.plot(forecast_dates, forecast_akbnk, label='90 Day Forecast', color='red')
plt.title("AKBNK - ARIMA Forecast (Log Transformed)")
plt.xlabel("Date")
plt.ylabel("Price (TL)")
plt.legend()
plt.tight_layout()
plt.show()

diff_log_akbnk = log_akbnk.diff().dropna()
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
plot_acf(diff_log_akbnk, ax=axes[0], lags=30)
axes[0].set_title("AKBNK - ACF (Log + 1. Fark)")
plot_pacf(diff_log_akbnk, ax=axes[1], lags=30, method='ywm')
axes[1].set_title("AKBNK - PACF (Log + 1. Fark)")
plt.tight_layout()
plt.show()


In [ ]:
models = {'THYAO': model_thy, 'AKBNK.IS': model_akbnk,  'ARCLK.IS': model_arclk}
for name, m in models.items():
    print(f"{name}  →  ARIMA{m.order} ,  AIC = {m.aic():.2f}, BIC = {m.bic():.2f}")


**Ljung-Box**

In [ ]:

residuals = model_thy.resid()

ljung_result = acorr_ljungbox(residuals, lags=[10], return_df=True)
print("thy:",ljung_result)

In [ ]:
residuals = model_akbnk.resid()

ljung_result = acorr_ljungbox(residuals, lags=[10], return_df=True)
print("akbnk:",ljung_result)

In [ ]:
residuals = model_arclk.resid()

ljung_result = acorr_ljungbox(residuals, lags=[10], return_df=True)
print("arclk:",ljung_result)

##  GARCH (1,1)

### Volatility Analysis (GARCH) for THYAO.IS Stock 

In [ ]:

thy_returns = returns['THYAO.IS'] * 100

model_thy_garch = arch_model(thy_returns, vol='Garch', p=1, q=1)
result_thy_garch = model_thy_garch.fit(disp='off')

print(result_thy_garch.summary())

# Volatilite Graph
plt.plot(result_thy_garch.conditional_volatility)
plt.title("THYAO - GARCH(1,1) Tahmini Volatilite")
plt.xlabel("Tarih")
plt.ylabel("Volatilite (%)")
plt.show()


residuals_thy = result_thy_garch.resid.dropna()

stats.probplot(residuals_thy, dist="norm", plot=plt)
plt.title("THYAO - GARCH Artıkları Q-Q Plot")
plt.grid(True)
plt.show()

lb_test = acorr_ljungbox(residuals_thy, lags=[10], return_df=True)
print("Ljung-Box Test Sonucu (THYAO):")
print(lb_test)


### Volatility Analysis (GARCH) for AKBNK.IS Stock

In [ ]:
akbnk_returns = returns['AKBNK.IS'] * 100
model_akbnk_garch = arch_model(akbnk_returns, vol='Garch', p=1, q=1)
result_akbnk_garch = model_akbnk_garch.fit(disp='off')
print(result_akbnk_garch.summary())

plt.plot(result_akbnk_garch.conditional_volatility)
plt.title("AKBNK - GARCH(1,1) Tahmini Volatilite")
plt.xlabel("Tarih")
plt.ylabel("Volatilite (%)")
plt.show()

residuals_akbnk = result_akbnk_garch.resid.dropna()
stats.probplot(residuals_akbnk, dist="norm", plot=plt)
plt.title("AKBANK - GARCH Artıkları Q-Q Plot")
plt.grid(True)
plt.show()

lb_test_akbnk = acorr_ljungbox(residuals_akbnk, lags=[10], return_df=True)
print("Ljung-Box Test Sonucu (AKBNK):")
print(lb_test_akbnk)


### Volatility Analysis (GARCH) for ARCLK.IS Stock 

In [ ]:
arclk_returns = returns['ARCLK.IS'] * 100

model_arclk_garch = arch_model(arclk_returns, vol='Garch', p=1, q=1)
result_arclk_garch = model_arclk_garch.fit(disp='off')

print(result_arclk_garch.summary())

plt.plot(result_arclk_garch.conditional_volatility)
plt.title("ARCLK - GARCH(1,1) Tahmini Volatilite")
plt.xlabel("Tarih")
plt.ylabel("Volatilite (%)")
plt.show()

residuals_arclk = result_arclk_garch.resid.dropna()
stats.probplot(residuals_arclk, dist="norm", plot=plt)
plt.title("ARCLK - GARCH Artıkları Q-Q Plot")
plt.grid(True)
plt.show()

lb_test_arclk = acorr_ljungbox(residuals_arclk, lags=[10], return_df=True)
print("Ljung-Box Test Sonucu (ARCLK):")
print(lb_test_arclk)

In [ ]:
garch_models = {
    'THYAO': result_thy_garch,
    'AKBNK': result_akbnk_garch,
    'ARCLK': result_arclk_garch
}

for name, res in garch_models.items():
    print(f"\n{name}  GARCH(1,1) Parametreleri")
    print(res.params)


## DIAGNOSTIC EXAMINATION OF RESIDUALS

### THYAO - Histogram and QQPlot of GARCH Model Residuals 

In [ ]:
plt.hist(result_thy_garch.std_resid, bins=30)
plt.title("Standardized Residuals Histogram - THYAO")
plt.show()

sm.qqplot(result_thy_garch.std_resid, line='s')
plt.title("QQ Plot - THYAO GARCH Residuals")
plt.show()

### ARCLK - Histogram and QQPlot of GARCH Model Residuals

In [ ]:
plt.hist(result_arclk_garch.std_resid, bins=30)
plt.title("Standardized Residuals Histogram - ARCLK")
plt.show()

sm.qqplot(result_arclk_garch.std_resid, line='s')
plt.title("QQ Plot - ARCLK GARCH Residuals")
plt.show()

### AKBNK - Histogram and QQPlot of GARCH Model Residuals

In [ ]:
plt.hist(result_akbnk_garch.std_resid, bins=30)
plt.title("Standardized Residuals Histogram - AKBNK")
plt.show()

sm.qqplot(result_akbnk_garch.std_resid, line='s')
plt.title("QQ Plot - AKBNK GARCH Residuals")
plt.show()

##  PORTFOLIO PERFORMANCE: COMPARISON OF FORECAST AND ACTUAL VALUES 

In [ ]:
investment = 1_000_000
weights = [1/3, 1/3, 1/3]
last_prices = data.iloc[-1]

shares = {
    'THYAO.IS': (weights[0] * investment) / last_prices['THYAO.IS'],
    'AKBNK.IS': (weights[1] * investment) / last_prices['AKBNK.IS'],
    'ARCLK.IS': (weights[2] * investment) / last_prices['ARCLK.IS'],
}


forecast_dates = pd.date_range(start=last_prices.name + timedelta(days=1), periods=90, freq='B')

portfolio_predicted_value = (
    forecast_thy * shares['THYAO.IS'] +
    forecast_akbnk * shares['AKBNK.IS'] +
    forecast_arclk * shares['ARCLK.IS']
)

print("forecast_dates:", forecast_dates)
print("kac:", len(forecast_dates))
print("portfolio_predicted_value:", portfolio_predicted_value[:5])



In [ ]:

stocks = ['THYAO.IS',
          'AKBNK.IS',
          'ARCLK.IS']

start_date = '2024-12-31'
end_date   = '2025-05-31'

raw_data = yf.download(stocks, start=start_date, end=end_date, auto_adjust=False)

if isinstance(raw_data.columns, pd.MultiIndex):
    adj_close = raw_data['Adj Close'].dropna()
else:
    adj_close = raw_data[['Adj Close']].dropna()

print(adj_close.head())




In [ ]:
real_portfolio_value = (
    adj_close['THYAO.IS'] * shares['THYAO.IS'] +
    adj_close['AKBNK.IS'] * shares['AKBNK.IS'] +
    adj_close['ARCLK.IS'] * shares['ARCLK.IS']
)

print(real_portfolio_value.head())


In [ ]:
plt.figure(figsize=(12, 6))


plt.plot(forecast_dates, portfolio_predicted_value, label='Tahmini Portföy Değeri (ARIMA)', color='red')
plt.plot(real_portfolio_value.index, real_portfolio_value, label='Gerçek Portföy Değeri', color='green')

plt.title("Portfolio Performance: Forecast vs. Actual (January 2025 - 90 Days)")
plt.xlabel("Date")
plt.ylabel("Portfolio Value (TL)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:

table = []
for w, s in zip(weights, stocks):
    alloc_tl = w * investment
    lot      = alloc_tl / last_prices[s]
    table.append([s,
                  round(last_prices[s],2),
                  round(alloc_tl,0),
                  round(lot,2)])

portfolio_df = pd.DataFrame(table,
            columns=['Share', 'Price (TL)', 'Investment Amount (TL)', 'Purchased Lot Quantity'])
print("\n=== Portfolio Purchase Summary ===")
print(portfolio_df.to_string(index=False))


**Forecasting Model**

The predicted value curve (red line) shows a clear upward trend. This indicates that the ARIMA model tends to continue the trend in historical data and assumes that the market will continue to move upwards.



**Actual Performance**

The graph shows that in early January, the portfolio value largely coincides with the forecast; however, Deviations begin in mid-February. Although there was a short-lived spike in early March, the portfolio value started to decline significantly in April. At the end of May, the portfolio value decreased by approximately 15% to TL 850,000.

## Results and Discussion


**Forecasting Performance Assessment**

The analysis of the 90-day price forecasts generated by the ARIMA model revealed varying degrees of accuracy across different time horizons

In the short run (until mid-January), the ARIMA forecasts closely matched the actual historical price data. However, the model's predictive capability significantly failed in the medium and long run. Especially after March, forecasting performance weakened, and the model proved incapable of predicting the sudden, sharp price changes observed in the portfolio components (THYAO, AKBNK, ARCLK).

**Analytical Implications and Methodological Development Proposal**

These results provide a critical insight: financial time series are highly sensitive not only to the underlying trend but also to volatility, seasonality, and exogenous shocks.

For future studies, we propose that combining established time-series methods like ARIMA with volatility-oriented models, such as GARCH  will yield more robust and reliable forecasting results.